In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import os
from variableUtils import *
from Utils import *
from ClassUtils import *
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image
from reportlab.lib import colors
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.platypus import Paragraph, Spacer, KeepTogether, KeepInFrame
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule
import json
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
import sys
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(sns.__version__)


(841.68, 1190.8799999999999)
0.13.2


In [2]:
colSupervisorData = "supervisor_data"
colStudentData = "student_data"
fullnamedict = {}
weightDict = {}
cutoffDict = {}
criticalDict = {}

fullnamedict["DDI-DDS"]={"id": 27, "key": "DDI-DDS", "name": "Dental Dam Isolation (DDS)", "group": "Miscellaneous", 
"fields": {   "MC1": "Nasal airway open and oral cavity covered",
              "MC2": "Chosen clamp fits securely around tooth",
              "MC3": "Clamp correctly secured with floss, floss is secured and contained",
              "MC4": "Clamp handling: wings and beaks exposed, no floss caught in beaks", 
              "MC5": "Accessory retention appropriately placed (if applicable)",
              "MC6": "Dam secured to frame, frame correctly oriented",
              "MC7": "Appropriate number of teeth isolated",
              "MC8": "Dam flossed interproximally, including distal to the clamp",
              "MC9": "Dam inverted around all teeth", 
              "MC10": "No large tears"}}
# highest weighted items are critical
weightDict["DDI-DDS"] = {"MC1": 19, "MC2": 2, "MC3": 19, "MC4": 19, "MC5": 19, "MC6": 19, "MC7": 19, "MC8": 2, "MC9": 2, "MC10": 19}
criticalDict["DDI-DDS"] = {"MC1": "Yes", "MC2": "No", "MC3": "Yes", "MC4": "Yes", "MC5": "Yes", "MC6": "Yes", "MC7": "Yes", "MC8": "No", "MC9": "No", "MC10": "Yes"}
cutoffDict["DDI-DDS"] = 81


fullnamedict["SOR-DDS"] = {"id": 30, "key": "SOR-DDS", "name": "DDS1 SmOcc Resto", "group": "Miscellaneous",
"fields": {"MC1": "Composite handling: Homogenous",
            "MC2": "Surface finish smooth",
            "MC3": "Margins: No deficiency",
            "MC4": "Margins: No excess",
            "MC5": "Tooth morphology and contour resembles anatomy",
            "MC6": "Adjacent teeth/surfaces: No iatrogenic damage"}}
weightDict["SOR-DDS"] = {"MC1": 18, "MC2": 2, "MC3": 18, "MC4": 2, "MC5": 1, "MC6": 18}
# the 18 weight items are critical others are not
criticalDict["SOR-DDS"] = {"MC1": "Yes", "MC2": "No", "MC3": "Yes", "MC4": "No", "MC5": "No", "MC6": "Yes"}
cutoffDict["SOR-DDS"] = 69


fullnamedict["SOP-DDS"] = {"id": 29, "key": "SOP-DDS", "name": "DDS1 SmOcc Prep", "group": "Miscellaneous",
"fields": {"MC1": "Selective caries removal demonstrated: pulpal",
            "MC2": "Selective caries removal demonstrated: peripheral surfaces",
            "MC3": "DEJ is caries free",
            "MC4": "Cavity outline reflects extent of caries", 
            "MC5": "Cavity depth reflects extent of caries", 
            "MC6": "Cavosurface margins supported?", 
            "MC7": "Internal surfaces smooth", 
            "MC8": "Adjacent teeth/surfaces: no iatrogenic damage"}}
weightDict["SOP-DDS"] = {"MC1": 7, "MC2": 7, "MC3": 7, "MC4": 7, "MC5": 7, "MC6": 7, "MC7": 2, "MC8": 7}
# the 7 weight items are critical others are not
criticalDict["SOP-DDS"] = {"MC1": "Yes", "MC2": "Yes", "MC3": "Yes", "MC4": "Yes", "MC5": "Yes", "MC6": "Yes", "MC7": "No", "MC8": "Yes"}
cutoffDict["SOP-DDS"] = 86


fullnamedict["FS-DDS"] = {"id": 28, "key": "FS-DDS", "name": "DDS1 FS", "group": "Miscellaneous", 
"fields": {"MC1": "Correct surface preparation prior to placement", 
           "MC2": "Neat placement in fissure system"}}
cutoffDict["FS-DDS"] = 50
criticalDict["FS-DDS"] = {"MC1": "NA", "MC2": "NA"}
weightDict["FS-DDS"] = {"MC1": 1, "MC2": 1}


fullnamedict["161"] = {"id": 26, "key": "161", "name": "Fissure and / or Tooth Surface Sealing", "group": "Preventive, Prophylactic and Bleaching Services", 
"fields": {"MC1": "Identifies material based on patient history and tooth factors.", 
           "MC2": "Isolates the tooth/teeth according to the patient and material choice to obtain moisture control", 
           "MC3": "Prepares tooth surface specific to material selected", 
           "MC4": "Applies sealant to all required pits and fissures and material is retained on the tooth surface.", 
           "MC5": "The sealant is finished (no overextension and overfilling, material is free of bubbles and voids the surface is smooth, the margins are undetected and is not interfering with occlusion)"}}
criticalDict["161"] = {"MC1": "NA", "MC2": "NA", "MC3": "NA", "MC4": "NA", "MC5": "NA"}
# weightDict["161"] = {"MC1": 1, "MC2": 1, "MC3": 1, "MC4": 1, "MC5": 1}
cutoffDict["161"] = 50

fullnamedict["DDI-BOH"] = {"id": 25, "key": "DDI-BOH", "name": "Dental Dam Isolation (BOH)", "group": "Miscellaneous", 
                           "fields": {"MC1": "Clamp selected fits tightly around tooth", 
                                      "MC2": "Stable Clamp", 
                                      "MC3": "Clamp secured with dental floss", 
                                      "MC4": "Hole size and spacing correlates to the area being isolated", 
                                      "MC5": "Dam flossed through interproximal spaces", 
                                      "MC6": "Dam inverted in operative area", 
                                      "MC7": "Frame oriented comfortably around patient's chin", 
                                      "MC8": "Patient's airway not restricted", 
                                      "MC9": "Opposing arch covered", 
                                      "MC10": "No tears or defects in the dam that could compromise treatment"}}
# all items are NA critical
criticalDict["DDI-BOH"] = {"MC1": "NA", "MC2": "NA", "MC3": "NA", "MC4": "NA", "MC5": "NA", "MC6": "NA", "MC7": "NA", "MC8": "NA", "MC9": "NA", "MC10": "NA"}
# weightDict["DDI-BOH"] = {"MC1": 1, "MC2": 1, "MC3": 1, "MC4": 1, "MC5": 1, "MC6": 1, "MC7": 1, "MC8": 1, "MC9": 1, "MC10": 1}
cutoffDict["DDI-BOH"] = 50


In [ ]:
workbookpath = '2025\DDS1\DDS1 2025 large occlusal assessment.xlsx'
folder, file ,ext = getFolderandFileName(workbookpath)
sheetname = 'Sheet1'
# sheetname = '2025-03-28'
# sheetname = '2025-04-03'
# sheetname = '2025-04-10'
# sheetname = '2025-04-11'

# sheetname = '2025-04-17'
# sheetname = 'all_data_flat'
df = pd.read_excel(workbookpath, sheet_name=sheetname)
df.rename(columns = {'student_number': 'student_id'}, inplace = True)
df.sort_values(by = ["student_id"], inplace = True)

commonCols = ["student_id", "student_name", "assessor_name", "date", "cohort", "type", "student_feedback", "assessor_feedback", "clinical_incident", "assessor_positioning_MC1", "assessor_positioning_MC2"]
rubricCols = ['time_mgmt'	,'entrustment'	,'communication'	,'professionalism'	]
beforeCols = ['student_id',	'student_name',	'assessor_name',	'date',	'cohort',	'type']
pecCols = ['assessor_positioning_MC1',	'assessor_positioning_MC2']
feedbackCols = ['student_feedback',	'assessor_feedback',	'clinical_incident']
# df[colSupervisorData] = df[colSupervisorData].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
# df[colStudentData] = df[colStudentData].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
# for i, col in enumerate(df.columns):
#     print(i, col)
# in student id replace 6 with 1357364 and 15 with 1353015
df['student_id'] = df['student_id'].replace(6, 1357364)
df['student_id'] = df['student_id'].replace(15, 1353015)


display(df.head(5))

,assessment_id,student_name,student_id,assessor_name,date,cohort,subject,type,student_submitted,assessor_submitted,student_feedback,student_DDI-DDS_MC1,student_DDI-DDS_MC2,student_DDI-DDS_MC3,student_DDI-DDS_MC4,student_DDI-DDS_MC5,student_DDI-DDS_MC6,student_DDI-DDS_MC7,student_DDI-DDS_MC8,student_DDI-DDS_MC9,student_SOP-DDS_MC1,student_SOP-DDS_MC2,student_SOP-DDS_MC3,student_SOP-DDS_MC4,student_SOP-DDS_MC5,student_SOP-DDS_MC6,student_SOP-DDS_MC7,student_SOP-DDS_MC8,student_SOR-DDS_MC1,student_SOR-DDS_MC2,student_SOR-DDS_MC3,student_SOR-DDS_MC4,student_SOR-DDS_MC5,student_SOR-DDS_MC6,student_DDI-DDS_MC10,time_mgmt,entrustment,communication,professionalism,assessor_feedback,clinical_incident,assessor_DDI-DDS_MC1,assessor_DDI-DDS_MC2,assessor_DDI-DDS_MC3,assessor_DDI-DDS_MC4,assessor_DDI-DDS_MC5,assessor_DDI-DDS_MC6,assessor_DDI-DDS_MC7,assessor_DDI-DDS_MC8,assessor_DDI-DDS_MC9,assessor_SOP-DDS_MC1,assessor_SOP-DDS_MC2,assessor_SOP-DDS_MC3,assessor_SOP-DDS_MC4,assessor_SOP-DDS_MC5,assessor_SOP-DDS_MC6,assessor_SOP-DDS_MC7,assessor_SOP-DDS_MC8,assessor_SOR-DDS_MC1,assessor_SOR-DDS_MC2,assessor_SOR-DDS_MC3,assessor_SOR-DDS_MC4,assessor_SOR-DDS_MC5,assessor_SOR-DDS_MC6,assessor_DDI-DDS_MC10,clinical_incident_radio,assessor_positioning_MC1,assessor_positioning_MC2,student_FS-DDS_MC1,student_FS-DDS_MC2,assessor_FS-DDS_MC1,assessor_FS-DDS_MC2
98,3888,Anna Pham,1357364,Audrey Choo,2025-04-11,DDS1,DENT90141,Simulation,True,True,I found the dental dam this session much more ...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Yes,2,2,1,2,Find a band to ensure eyewear doesn’t keep sli...,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Yes,No,Yes,Yes,Not Assessed,Not Assessed,Not Reviewed,Not Reviewed
67,3862,Jenny Feng,1353015,Miranda Hua,2025-04-11,DDS1,DENT90141,Simulation,True,True,"When punching holes, punch lower in dental dam...",No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,Yes,Yes,Yes,Yes,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Yes,2,1,1,1,"Dental dam 44-47 - minor interproximal tears, ...",NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Yes,No,Yes,Yes,NaN,NaN,NaN,NaN
12,3796,Paula Yu,839975,Ari Barr,2025-04-11,DDS1,DENT90141,Simulation,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,Demo advice was taken on board well,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Yes,No,Yes,Yes,NaN,NaN,NaN,NaN
79,3777,Aaron Khoo,993809,Fammy Liem,2025-04-11,DDS1,DENT90141,Simulation,True,True,"No issues with Dental Dam Isolation overall, e...",Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,2,2,2,Overextended distally and buccally- realised. ...,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,NaN,NaN,NaN,NaN
95,3774,Lorenzo Gullone,1079529,Roland Chong,2025-04-11,DDS1,DENT90141,Simulation,True,True,"I struggled to get my dental dam on, and was u...",Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,No,No,Yes,Yes,Yes,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,No,4,3,2,2,Tear on the buccal of 45. Dam not sealing well...,NaN,No,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,Yes,No,Yes,Yes,NaN,NaN,NaN,NaN


In [154]:
sopCols = [col for col in df.columns if 'SOP' in col]
sorCols = [col for col in df.columns if 'SOR' in col]
fsCols = [col for col in df.columns if 'FS' in col]
# _161Cols = [col for col in df.columns if '161' in col]
ddCols = [col for col in df.columns if 'DDI-DDS' in col]
mcCols =   ddCols + fsCols + sopCols + sorCols
mcCols = [col for col in mcCols if col in df.columns]
assessormcCols = [col for col in mcCols if 'assessor' in col]
scoreCols = []
for key in weightDict.keys():
    df[f'{key} total'] = 0  # create the column with default 0
    df[f'{key} score'] = 0  # create the column with default 0
    df[f'{key} cutoff'] = cutoffDict.get(key, pd.NA) # create the column with default 0
    df[f'{key} pass'] = '' # create the column with default 0
    df[f'{key} # Yes'] = 0  # create the column with default 0
    df[f'{key} # No'] = 0  # create the column with default 0
    scoreCols.append(f'{key} # Yes')
    scoreCols.append(f'{key} # No')
    scoreCols.append(f'{key} score')
    scoreCols.append(f'{key} cutoff')
    scoreCols.append(f'{key} pass')




df.replace({'Not Reviewed': 'No'}, inplace=True)
for i, row in df.iterrows():
    denominator = {}
    for col in assessormcCols:
        key = col.split('_')[1]
        qno = col.split('_')[2]
        if key in weightDict.keys():
            if row[col] == 'Yes':
                df.at[i, f'{key} total'] += weightDict[key][qno]
                df.at[i, f'{key} # Yes'] += 1
                denominator[key] = denominator.get(key, 0) + weightDict[key][qno]
            elif row[col] == 'No':
                df.at[i, f'{key} total'] += 0
                df.at[i, f'{key} # No'] += 1
                denominator[key] = denominator.get(key, 0) + weightDict[key][qno]
            elif row[col] == 'Not Reviewed':
                df.at[i, f'{key} total'] += 0
                denominator[key] = denominator.get(key, 0) + 0
            else:
                df.at[i, f'{key} total'] += 0
                denominator[key] = denominator.get(key, 0) + 0
            print(f"Row {i}: {col} = {df.at[i, f'{key} total']}")
    print(f"Row {i}: {denominator}")

    for key in denominator.keys():
        if denominator[key] > 0:
            df.at[i, f'{key} score'] = round(df.at[i, f'{key} total'] / denominator[key] * 100, 2)
        
            df.at[i, f'{key} pass'] = 'Pass' if df.at[i, f'{key} score'] >= cutoffDict.get(key, 0) else 'Fail'
        
        elif denominator[key] == 0:
            df.at[i, f'{key} score'] = pd.NA
            df.at[i, f'{key} pass'] = pd.NA
            # df.at[i, f'{key} score'] = 0
    # display(df.iloc[i:i+1])
    # break

display(df.head(5))

Row 98: assessor_DDI-DDS_MC1 = 19
Row 98: assessor_DDI-DDS_MC2 = 21
Row 98: assessor_DDI-DDS_MC3 = 40
Row 98: assessor_DDI-DDS_MC4 = 59
Row 98: assessor_DDI-DDS_MC5 = 78
Row 98: assessor_DDI-DDS_MC6 = 97
Row 98: assessor_DDI-DDS_MC7 = 116
Row 98: assessor_DDI-DDS_MC8 = 118
Row 98: assessor_DDI-DDS_MC9 = 120
Row 98: assessor_DDI-DDS_MC10 = 139
Row 98: assessor_FS-DDS_MC1 = 0
Row 98: assessor_FS-DDS_MC2 = 0
Row 98: assessor_SOP-DDS_MC1 = 7
Row 98: assessor_SOP-DDS_MC2 = 14
Row 98: assessor_SOP-DDS_MC3 = 21
Row 98: assessor_SOP-DDS_MC4 = 28
Row 98: assessor_SOP-DDS_MC5 = 35
Row 98: assessor_SOP-DDS_MC6 = 42
Row 98: assessor_SOP-DDS_MC7 = 44
Row 98: assessor_SOP-DDS_MC8 = 51
Row 98: assessor_SOR-DDS_MC1 = 0
Row 98: assessor_SOR-DDS_MC2 = 0
Row 98: assessor_SOR-DDS_MC3 = 0
Row 98: assessor_SOR-DDS_MC4 = 0
Row 98: assessor_SOR-DDS_MC5 = 0
Row 98: assessor_SOR-DDS_MC6 = 0
Row 98: {'DDI-DDS': 139, 'FS-DDS': 2, 'SOP-DDS': 51, 'SOR-DDS': 59}
Row 67: assessor_DDI-DDS_MC1 = 19
Row 67: assessor_DDI

,assessment_id,student_name,student_id,assessor_name,date,cohort,subject,type,student_submitted,assessor_submitted,student_feedback,student_DDI-DDS_MC1,student_DDI-DDS_MC2,student_DDI-DDS_MC3,student_DDI-DDS_MC4,student_DDI-DDS_MC5,student_DDI-DDS_MC6,student_DDI-DDS_MC7,student_DDI-DDS_MC8,student_DDI-DDS_MC9,student_SOP-DDS_MC1,student_SOP-DDS_MC2,student_SOP-DDS_MC3,student_SOP-DDS_MC4,student_SOP-DDS_MC5,student_SOP-DDS_MC6,student_SOP-DDS_MC7,student_SOP-DDS_MC8,student_SOR-DDS_MC1,student_SOR-DDS_MC2,student_SOR-DDS_MC3,student_SOR-DDS_MC4,student_SOR-DDS_MC5,student_SOR-DDS_MC6,student_DDI-DDS_MC10,time_mgmt,entrustment,communication,professionalism,assessor_feedback,clinical_incident,assessor_DDI-DDS_MC1,assessor_DDI-DDS_MC2,assessor_DDI-DDS_MC3,assessor_DDI-DDS_MC4,assessor_DDI-DDS_MC5,assessor_DDI-DDS_MC6,assessor_DDI-DDS_MC7,assessor_DDI-DDS_MC8,assessor_DDI-DDS_MC9,assessor_SOP-DDS_MC1,assessor_SOP-DDS_MC2,assessor_SOP-DDS_MC3,assessor_SOP-DDS_MC4,assessor_SOP-DDS_MC5,assessor_SOP-DDS_MC6,assessor_SOP-DDS_MC7,assessor_SOP-DDS_MC8,assessor_SOR-DDS_MC1,assessor_SOR-DDS_MC2,assessor_SOR-DDS_MC3,assessor_SOR-DDS_MC4,assessor_SOR-DDS_MC5,assessor_SOR-DDS_MC6,assessor_DDI-DDS_MC10,clinical_incident_radio,assessor_positioning_MC1,assessor_positioning_MC2,student_FS-DDS_MC1,student_FS-DDS_MC2,assessor_FS-DDS_MC1,assessor_FS-DDS_MC2,DDI-DDS total,DDI-DDS score,DDI-DDS cutoff,DDI-DDS pass,DDI-DDS # Yes,DDI-DDS # No,SOR-DDS total,SOR-DDS score,SOR-DDS cutoff,SOR-DDS pass,SOR-DDS # Yes,SOR-DDS # No,SOP-DDS total,SOP-DDS score,SOP-DDS cutoff,SOP-DDS pass,SOP-DDS # Yes,SOP-DDS # No,FS-DDS total,FS-DDS score,FS-DDS cutoff,FS-DDS pass,FS-DDS # Yes,FS-DDS # No
98,3888,Anna Pham,1357364,Audrey Choo,2025-04-11,DDS1,DENT90141,Simulation,True,True,I found the dental dam this session much more ...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Yes,2,2,1,2,Find a band to ensure eyewear doesn’t keep sli...,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,Yes,No,Yes,Yes,Not Assessed,Not Assessed,No,No,139,100.00,81,Pass,10,0,0,0.0,69,Fail,0,6,51,100.00,86,Pass,8,0,0,0.0,50,Fail,0,2
67,3862,Jenny Feng,1353015,Miranda Hua,2025-04-11,DDS1,DENT90141,Simulation,True,True,"When punching holes, punch lower in dental dam...",No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,Yes,Yes,Yes,Yes,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Yes,2,1,1,1,"Dental dam 44-47 - minor interproximal tears, ...",NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,No,No,No,No,No,No,Yes,No,Yes,Yes,NaN,NaN,NaN,NaN,139,100.00,81,Pass,10,0,0,0.0,69,Fail,0,6,44,86.27,86,Pass,7,1,0,NaN,50,<NA>,0,0
12,3796,Paula Yu,839975,Ari Barr,2025-04-11,DDS1,DENT90141,Simulation,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,Demo advice was taken on board well,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,Yes,No,Yes,Yes,NaN,NaN,NaN,NaN,139,100.00,81,Pass,10,0,0,0.0,69,Fail,0,6,51,100.00,86,Pass,8,0,0,NaN,50,<NA>,0,0
79,3777,Aaron Khoo,993809,Fammy Liem,2025-04-11,DDS1,DENT90141,Simulation,True,True,"No issues with Dental Dam Isolation overall, e...",Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,2,2,2,Overextended distally and buccally- realised. ...,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,NaN,NaN,NaN,NaN,139,100.00,81,Pass,10,0,59,100.0,69,Pass,6,0,51,100.00,86,Pass,8,0,0,NaN,50,<NA>,0,0
95,3774,Lorenzo Gullone,1079529,Roland Chong,2025-04-11,DDS1,DENT90141,Simulation,True,True,"I struggled to get my dental dam on, and was u...",Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,No,No,Yes,Yes,Yes,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Asses

In [155]:
newAssessormcCols = []
for col in assessormcCols:
    new_col = col.replace('assessor_', '')
    newAssessormcCols.append(new_col)
    df.rename(columns={col: new_col}, inplace=True)

sortedKeys = ['DDI-DDS', 'FS-DDS', 'SOP-DDS', 'SOR-DDS']
# need rearranging of columns keys together
rearrangedCols = []
rearrangedCols += beforeCols
# DDI-DDS FS-DDS SOP-DDS SOR-DDS mcCols
rearrangedCols += newAssessormcCols
# rearrange scoreCols such that DDI-DDS, FS-DDS, SOP-DDS, SOR-DDS keys are in order
sortedScoreCols = []
for key in sortedKeys:
    for col in scoreCols:
        if key in col:
            sortedScoreCols.append(col)
rearrangedCols += sortedScoreCols

rearrangedCols += rubricCols
rearrangedCols += feedbackCols
print(rearrangedCols)
df = df[rearrangedCols]

# remove assessor from the column names that are mcCols

# df.replace({'Yes': 1, 'No': 0, 'Not Reviewed': 'NA'}, inplace=True)
display(df.head(5))
# save the dataframe to a new excel file with keys columns color coded
# craete color coding for the columns
# for col in df.columns:
#     print(col)

['student_id', 'student_name', 'assessor_name', 'date', 'cohort', 'type', 'DDI-DDS_MC1', 'DDI-DDS_MC2', 'DDI-DDS_MC3', 'DDI-DDS_MC4', 'DDI-DDS_MC5', 'DDI-DDS_MC6', 'DDI-DDS_MC7', 'DDI-DDS_MC8', 'DDI-DDS_MC9', 'DDI-DDS_MC10', 'FS-DDS_MC1', 'FS-DDS_MC2', 'SOP-DDS_MC1', 'SOP-DDS_MC2', 'SOP-DDS_MC3', 'SOP-DDS_MC4', 'SOP-DDS_MC5', 'SOP-DDS_MC6', 'SOP-DDS_MC7', 'SOP-DDS_MC8', 'SOR-DDS_MC1', 'SOR-DDS_MC2', 'SOR-DDS_MC3', 'SOR-DDS_MC4', 'SOR-DDS_MC5', 'SOR-DDS_MC6', 'DDI-DDS # Yes', 'DDI-DDS # No', 'DDI-DDS score', 'DDI-DDS cutoff', 'DDI-DDS pass', 'FS-DDS # Yes', 'FS-DDS # No', 'FS-DDS score', 'FS-DDS cutoff', 'FS-DDS pass', 'SOP-DDS # Yes', 'SOP-DDS # No', 'SOP-DDS score', 'SOP-DDS cutoff', 'SOP-DDS pass', 'SOR-DDS # Yes', 'SOR-DDS # No', 'SOR-DDS score', 'SOR-DDS cutoff', 'SOR-DDS pass', 'time_mgmt', 'entrustment', 'communication', 'professionalism', 'student_feedback', 'assessor_feedback', 'clinical_incident']


,student_id,student_name,assessor_name,date,cohort,type,DDI-DDS_MC1,DDI-DDS_MC2,DDI-DDS_MC3,DDI-DDS_MC4,DDI-DDS_MC5,DDI-DDS_MC6,DDI-DDS_MC7,DDI-DDS_MC8,DDI-DDS_MC9,DDI-DDS_MC10,FS-DDS_MC1,FS-DDS_MC2,SOP-DDS_MC1,SOP-DDS_MC2,SOP-DDS_MC3,SOP-DDS_MC4,SOP-DDS_MC5,SOP-DDS_MC6,SOP-DDS_MC7,SOP-DDS_MC8,SOR-DDS_MC1,SOR-DDS_MC2,SOR-DDS_MC3,SOR-DDS_MC4,SOR-DDS_MC5,SOR-DDS_MC6,DDI-DDS # Yes,DDI-DDS # No,DDI-DDS score,DDI-DDS cutoff,DDI-DDS pass,FS-DDS # Yes,FS-DDS # No,FS-DDS score,FS-DDS cutoff,FS-DDS pass,SOP-DDS # Yes,SOP-DDS # No,SOP-DDS score,SOP-DDS cutoff,SOP-DDS pass,SOR-DDS # Yes,SOR-DDS # No,SOR-DDS score,SOR-DDS cutoff,SOR-DDS pass,time_mgmt,entrustment,communication,professionalism,student_feedback,assessor_feedback,clinical_incident
98,1357364,Anna Pham,Audrey Choo,2025-04-11,DDS1,Simulation,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,10,0,100.00,81,Pass,0,2,0.0,50,Fail,8,0,100.00,86,Pass,0,6,0.0,69,Fail,2,2,1,2,I found the dental dam this session much more ...,Find a band to ensure eyewear doesn’t keep sli...,NaN
67,1353015,Jenny Feng,Miranda Hua,2025-04-11,DDS1,Simulation,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,NaN,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,No,No,No,No,No,No,10,0,100.00,81,Pass,0,0,NaN,50,<NA>,7,1,86.27,86,Pass,0,6,0.0,69,Fail,2,1,1,1,"When punching holes, punch lower in dental dam...","Dental dam 44-47 - minor interproximal tears, ...",NaN
12,839975,Paula Yu,Ari Barr,2025-04-11,DDS1,Simulation,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No,10,0,100.00,81,Pass,0,0,NaN,50,<NA>,8,0,100.00,86,Pass,0,6,0.0,69,Fail,2,2,2,2,NaN,Demo advice was taken on board well,NaN
79,993809,Aaron Khoo,Fammy Liem,2025-04-11,DDS1,Simulation,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,10,0,100.00,81,Pass,0,0,NaN,50,<NA>,8,0,100.00,86,Pass,6,0,100.0,69,Pass,3,2,2,2,"No issues with Dental Dam Isolation overall, e...",Overextended distally and buccally- realised. ...,NaN
95,1079529,Lorenzo Gullone,Roland Chong,2025-04-11,DDS1,Simulation,No,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,NaN,NaN,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,No,No,No,8,2,72.66,81,Fail,0,0,NaN,50,<NA>,7,1,96.08,86,Pass,0,6,0.0,69,Fail,4,3,2,2,"I struggled to get my dental dam on, and was u...",Tear on the buccal of 45. Dam not sealing well...,NaN


In [156]:


# Define colors per key
colorMap = {
    'DDI-DDS': 'ADD8E6',   # Light Blue
    'FS-DDS': '90EE90',    # Light Green
    'SOP-DDS': 'FFD700',   # Gold
    'SOR-DDS': 'D9D2E9'    # Light Pink
}

# Load or prepare your dataframe (replace with your actual DataFrame)
# df = pd.read_csv('your_data.csv')  # example

# Save with color formatting
try:
    with pd.ExcelWriter(f'{folder}/formatted_output.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.sort_values(by = ["student_id"], inplace = True)
        df.to_excel(writer, index=False, sheet_name=sheetname)
        workbook = writer.book
        worksheet = writer.sheets[sheetname]

        # Iterate through columns and apply color fill
        for colIdx, colName in enumerate(df.columns, start=1):
            for key in sortedKeys:
                if colName.startswith(key) and 'pass' not in colName:
                    fill = PatternFill(start_color=colorMap[key], end_color=colorMap[key], fill_type='solid')
                    for row in range(2, len(df) + 2):  # skip header
                        worksheet.cell(row=row, column=colIdx).fill = fill
except FileNotFoundError:
    with pd.ExcelWriter(f'{folder}/formatted_output.xlsx', engine='openpyxl') as writer:
        df.sort_values(by = ["student_id"], inplace = True)
        df.to_excel(writer, index=False, sheet_name=sheetname)
        workbook = writer.book
        worksheet = writer.sheets[sheetname]

        # Iterate through columns and apply color fill
        for colIdx, colName in enumerate(df.columns, start=1):
            for key in sortedKeys:
                if colName.startswith(key) and 'pass' not in colName:
                    fill = PatternFill(start_color=colorMap[key], end_color=colorMap[key], fill_type='solid')
                    for row in range(2, len(df) + 2):  # skip header
                        worksheet.cell(row=row, column=colIdx).fill = fill
sys.exit()

SystemExit: 

In [ ]:
studentIds = df['student_id'].unique()
sectionDfDict = {}

for key in weightDict.keys():
        keyCols = [col for col in df.columns if key in col and 'student' not in col and 'MC' in col]
        metricCols = [ f'{key} # Yes', f'{key} # No', f'{key} total', f'{key} score', f'{key} cutoff', f'{key} pass']
        print(keyCols)
        print(metricCols)
        sectionDf = df[beforeCols + keyCols + pecCols + rubricCols + metricCols + feedbackCols]
        sectionDf.replace({'Yes': 1, 'No': 0, 'Not Reviewed': 'NA'}, inplace=True)
        # remove assessor_ from the column names except for feedback columns
        sectionDf.columns = [col.replace('assessor_', '') if 'assessor_' in col and 'feedback' not in col and 'name' not in col else col for col in sectionDf.columns]
     #    sectionDf.columns = [col.replace(f'{key}_', '') if f'{key}_' in col else col for col in sectionDf.columns]
        for col in metricCols:
             sectionDf.rename(columns={col: col.replace(f'{key} ', '')}, inplace=True)

        print(sectionDf.columns)
        # remove rows with nan in key score column
        sectionDf = sectionDf[sectionDf[f'score'].notna()]
        sectionDf[f'cutoff'] = cutoffDict.get(key, 'N/A')
        sectionDf[f'pass'] = sectionDf[f'score'] >= cutoffDict.get(key, 0)
        # set N/A for key pass if key socre is N/A
        sectionDf.loc[sectionDf[f'score'].isna(), f'pass'] = pd.NA
        sectionDf.sort_values(by = [ "student_id", "date"], inplace = True)
        sectionDfDict[key] = sectionDf
        # display(sectionDf.head(5))
# display(sectionDf.head(5))
i=0
for key, sectiondf in sectionDfDict.items():
     print(key, sectiondf.shape)
     display(sectiondf.head(5))
     outfile = f"{folder}\\section wise scores {sheetname} NA as No.xlsx"
     try:
        with pd.ExcelWriter(outfile, engine= 'openpyxl', mode= 'a', if_sheet_exists='replace') as writer:
            sectiondf.to_excel(writer, index=False, sheet_name=key)
     except FileNotFoundError:
        with pd.ExcelWriter(outfile, engine='openpyxl') as writer:
            sectiondf.to_excel(writer, index=False, sheet_name=key)
     


# end the run here

sys.exit()

['assessor_DDI-DDS_MC1', 'assessor_DDI-DDS_MC2', 'assessor_DDI-DDS_MC3', 'assessor_DDI-DDS_MC4', 'assessor_DDI-DDS_MC5', 'assessor_DDI-DDS_MC6', 'assessor_DDI-DDS_MC7', 'assessor_DDI-DDS_MC8', 'assessor_DDI-DDS_MC9', 'assessor_DDI-DDS_MC10']
['DDI-DDS # Yes', 'DDI-DDS # No', 'DDI-DDS total', 'DDI-DDS score', 'DDI-DDS cutoff', 'DDI-DDS pass']
Index(['student_id', 'student_name', 'assessor_name', 'date', 'cohort', 'type',
       'MC1', 'MC2', 'MC3', 'MC4', 'MC5', 'MC6', 'MC7', 'MC8', 'MC9', 'MC10',
       'positioning_MC1', 'positioning_MC2', 'time_mgmt', 'entrustment',
       'communication', 'professionalism', '# Yes', '# No', 'total', 'score',
       'cutoff', 'pass', 'student_feedback', 'assessor_feedback',
       'clinical_incident'],
      dtype='object')
['assessor_SOR-DDS_MC1', 'assessor_SOR-DDS_MC2', 'assessor_SOR-DDS_MC3', 'assessor_SOR-DDS_MC4', 'assessor_SOR-DDS_MC5', 'assessor_SOR-DDS_MC6']
['SOR-DDS # Yes', 'SOR-DDS # No', 'SOR-DDS total', 'SOR-DDS score', 'SOR-DDS cutoff',

,student_id,student_name,assessor_name,date,cohort,type,MC1,MC2,MC3,MC4,MC5,MC6,MC7,MC8,MC9,MC10,positioning_MC1,positioning_MC2,time_mgmt,entrustment,communication,professionalism,# Yes,# No,total,score,cutoff,pass,student_feedback,assessor_feedback,clinical_incident
80,6,Anna Pham,Amy Fam,2025-04-17,DDS1,Simulation,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,2,1,1,2,8,2,101,0.73,0.81,False,I found that applying the dental dam this sess...,Ensure dam is tight over dam and is engaged on...,NaN
45,15,Jenny Feng,Dominic Ho,2025-04-17,DDS1,Simulation,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,2,2,2,2,9,1,120,0.86,0.81,True,Make sure that dental dam is not covering nasa...,Finishing and polishing without losing anatomy...,NaN
108,839975,Paula Yu,Aditi Bhide,2025-04-17,DDS1,Simulation,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1,0,2,2,2,2,9,1,120,0.86,0.81,True,NaN,Remember to mark the midline and count the num...,NaN
87,993809,Aaron Khoo,Jenny liu,2025-04-17,DDS1,Simulation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1,1,4,3,2,2,9,1,137,0.99,0.81,True,"Faster at putting on rubber dam, missed invers...","I agree with all your reflection comments, goo...",NaN
94,1079529,Lorenzo Gullone,Julie Taing,2025-04-17,DDS1,Simulation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,1,2,3,1,2,10,0,139,1.00,0.81,True,Dental dam went well today. No issues with tha...,Efficient placement of RD _x000D_\nLight off r...,NaN


SOR-DDS (124, 27)


,student_id,student_name,assessor_name,date,cohort,type,MC1,MC2,MC3,MC4,MC5,MC6,positioning_MC1,positioning_MC2,time_mgmt,entrustment,communication,professionalism,# Yes,# No,total,score,cutoff,pass,student_feedback,assessor_feedback,clinical_incident
80,6,Anna Pham,Amy Fam,2025-04-17,DDS1,Simulation,1.0,0.0,1.0,1.0,1.0,1.0,1,1,2,1,1,2,5,1,57,0.97,0.69,True,I found that applying the dental dam this sess...,Ensure dam is tight over dam and is engaged on...,NaN
45,15,Jenny Feng,Dominic Ho,2025-04-17,DDS1,Simulation,1.0,1.0,1.0,0.0,1.0,1.0,1,1,2,2,2,2,5,1,57,0.97,0.69,True,Make sure that dental dam is not covering nasa...,Finishing and polishing without losing anatomy...,NaN
108,839975,Paula Yu,Aditi Bhide,2025-04-17,DDS1,Simulation,1.0,0.0,1.0,1.0,1.0,1.0,1,0,2,2,2,2,5,1,57,0.97,0.69,True,NaN,Remember to mark the midline and count the num...,NaN
87,993809,Aaron Khoo,Jenny liu,2025-04-17,DDS1,Simulation,1.0,1.0,1.0,1.0,1.0,1.0,1,1,4,3,2,2,6,0,59,1.00,0.69,True,"Faster at putting on rubber dam, missed invers...","I agree with all your reflection comments, goo...",NaN
94,1079529,Lorenzo Gullone,Julie Taing,2025-04-17,DDS1,Simulation,1.0,1.0,1.0,1.0,1.0,1.0,0,1,2,3,1,2,6,0,59,1.00,0.69,True,Dental dam went well today. No issues with tha...,Efficient placement of RD _x000D_\nLight off r...,NaN


SOP-DDS (103, 29)


,student_id,student_name,assessor_name,date,cohort,type,MC1,MC2,MC3,MC4,MC5,MC6,MC7,MC8,positioning_MC1,positioning_MC2,time_mgmt,entrustment,communication,professionalism,# Yes,# No,total,score,cutoff,pass,student_feedback,assessor_feedback,clinical_incident
80,6,Anna Pham,Amy Fam,2025-04-17,DDS1,Simulation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,2,1,1,2,0,8,0,0.00,0.86,False,I found that applying the dental dam this sess...,Ensure dam is tight over dam and is engaged on...,NaN
45,15,Jenny Feng,Dominic Ho,2025-04-17,DDS1,Simulation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,2,2,2,2,0,8,0,0.00,0.86,False,Make sure that dental dam is not covering nasa...,Finishing and polishing without losing anatomy...,NaN
108,839975,Paula Yu,Aditi Bhide,2025-04-17,DDS1,Simulation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,2,2,2,2,0,8,0,0.00,0.86,False,NaN,Remember to mark the midline and count the num...,NaN
87,993809,Aaron Khoo,Jenny liu,2025-04-17,DDS1,Simulation,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,4,3,2,2,7,1,44,0.86,0.86,True,"Faster at putting on rubber dam, missed invers...","I agree with all your reflection comments, goo...",NaN
94,1079529,Lorenzo Gullone,Julie Taing,2025-04-17,DDS1,Simulation,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0,1,2,3,1,2,7,1,44,0.86,0.86,True,Dental dam went well today. No issues with tha...,Efficient placement of RD _x000D_\nLight off r...,NaN


SystemExit: 

In [ ]:
summaryCols = ["student_id", "student_name", "assessor_name", "date", "student_feedback", "assessor_feedback", "clinical_incident"]
for key in weightDict.keys():
    summaryCols.append(f'{key} total')
    summaryCols.append(f'{key} score')
    # summaryCols.append(f'{key} # Yes')
    # summaryCols.append(f'{key} # No')
    summaryCols.append(f'{key} cutoff')
    summaryCols.append(f'{key} pass')
studentSummaryDf = pd.DataFrame(columns = summaryCols)
# for key in weightDict.keys():
    # studentSummaryDf[f'{key} total'] = df.groupby('student_id')[f'{key} total'].sum()
    # studentSummaryDf[f'{key} cutoff'] = cutoffDict.get(key, 'N/A')
    # studentSummaryDf[f'{key} pass'] = studentSummaryDf[f'{key} score'] >= cutoffDict.get(key, 0)
    # set N/A for key pass if key socre is N/A
    
for id in studentIds:
    studentDf = df[df['student_id'] == id]
    # studentDf = studentDf[summaryCols]
    for  key in weightDict.keys():
        studentDf[f'{key} cutoff'] = cutoffDict.get(key, 'N/A')
        studentDf[f'{key} pass'] = studentDf[f'{key} score'] >= cutoffDict.get(key, 0)
        # set N/A for key pass if key socre is N/A
        studentDf.loc[studentDf[f'{key} score'].isna(), f'{key} pass'] = pd.NA
    studentSummaryDf = pd.concat([studentSummaryDf, studentDf[summaryCols]])
    # display(studentDf.head(5))
    # display(studentDf.head(5))
        
        # display(sectionDf.head(5))
    # break
studentSummaryDf.sort_values(by = ["student_id"], inplace = True)
display(studentSummaryDf)

,student_id,student_name,assessor_name,date,student_feedback,assessor_feedback,clinical_incident,DDI-DDS total,DDI-DDS score,DDI-DDS cutoff,DDI-DDS pass,SOR-DDS total,SOR-DDS score,SOR-DDS cutoff,SOR-DDS pass,SOP-DDS total,SOP-DDS score,SOP-DDS cutoff,SOP-DDS pass
0,839975,Paula Yu,Amy Fam,2025-04-03,I learned to understand that high speed burs c...,Wear loupes as part of donning PPE. Operator c...,NaN,0,NaN,0.81,NaN,0,NaN,0.69,NaN,30,0.59,0.86,False
1,839975,Paula Yu,Melinda Johansson,2025-03-28,It is a great learning opportunity. In particu...,Good effort,NaN,139,1.00,0.81,True,0,NaN,0.69,NaN,0,NaN,0.86,NaN
2,993809,Aaron Khoo,Vicky Thai,2025-04-03,Overall was smooth. When placing fissure seal ...,Good placement,Took one glove off to handle workbook but held...,0,NaN,0.81,NaN,0,NaN,0.69,NaN,0,NaN,0.86,NaN
3,993809,Aaron Khoo,Vicky Thai,2025-04-03,Initial access went towards periphery of cavit...,Cavity prep not smooth internally but managed ...,NaN,0,NaN,0.81,NaN,59,1.00,0.69,True,51,1.00,0.86,True
4,993809,Aaron Khoo,Miranda Hua,2025-03-28,Happy with single tooth isolation._x000D_\nTer...,Final task not assessed but good time manageme...,NaN,0,NaN,0.81,NaN,0,NaN,0.69,NaN,0,NaN,0.86,NaN
5,1079529,Lorenzo Gullone,Aditi Bhide,2025-04-03,"Overall, was happy with the application of sea...",Well placed,NaN,0,NaN,0.81,NaN,0,NaN,0.69,NaN,0,NaN,0.86,NaN
6,1079529,Lorenzo Gullone,Aditi Bhide,2025-04-03,"Apart from my access being too large, I was re...",Significantly overextended in all dimensions. ...,NaN,0,NaN,0.81,NaN,58,0.98,0.69,True,44,0.86,0.86,True
7,1079529,Lorenzo Gullone,Ling Teh,2025-03-28,I struggled to get the dam wrapped around each...,Task not completed due RD tears - to gradually...,NaN,0,NaN,0.81,NaN,0,NaN,0.69,NaN,0,NaN,0.86,NaN
10,1082382,Jason Li,Miranda Hua,2025-04-10,NaN,NaN,NaN,0,NaN,0.81,NaN,0,NaN,0.69,NaN,0,NaN,0.86,NaN
8,1082382,Jason Li,Miranda Hua,2025-04-10,NaN,NaN,NaN,0,NaN,0.81,NaN,0,NaN,0.69,NaN,0,NaN,0.86,NaN


In [ ]:
studentSummaryDf.to_excel(f"{folder}\\student summary scores {sheetname}.xlsx", index=False, sheet_name=sheetname)

## Report

In [ ]:
df.fillna('', inplace=True)
df['student_id'] = df['student_id'].astype(str)
pecItems  = ['positioning', 'Consent', 'Record', 'infection']
pecCols = [col for col in df.columns if 'positioning' in col or 'Consent' in col or 'Record' in col or 'infection' in col]
studentmcCols = [col for col in mcCols if 'student' in col]
assessormcCols = [col for col in mcCols if 'assessor' in col]
otherCols = commonCols
print(otherCols)
otherfullnamedict = {'student_name': 'Student Name', 'assessor_name': 'Assessor Name', 'date': 'Date', 'cohort': 'Cohort', 'subject': 'Subject', 'type': 'Type',
                     'time_mgmt': 'Time Management', 'communication': 'Communication', 'professionalism': 'Professionalism', 'entrustment': 'Entrustment',
                     'student_feedback': 'Student Feedback', 'assessor_feedback': 'Assessor Feedback', 'clinical_incident': 'Clinical Incident',
                     'patient_complexity': 'Complexity', 'Consent': 'Consent'}
mcColDict = {}
for col in mcCols:
    key = col.split('_')[1]
    if key not in mcColDict:
        mcColDict[key] = []
    mcColDict[key].append(col)
def get_items(row):
    items = []
    for key, cols in mcColDict.items():
        if any((not pd.isna(row[col])) and (row[col] not in ['', 'nan']) for col in cols):
            if key in pecItems:
                continue
            items.append(key)
    return items

df['Items'] = df.apply(get_items, axis=1)
display(df.head())

['student_id', 'student_name', 'assessor_name', 'date', 'cohort', 'type', 'student_feedback', 'assessor_feedback', 'clinical_incident', 'assessor_positioning_MC1', 'assessor_positioning_MC2']


,student_name,student_id,assessor_name,date,cohort,subject,type,time_mgmt,entrustment,communication,professionalism,assessor_feedback,clinical_incident,assessor_DDI-DDS_MC1,assessor_DDI-DDS_MC2,assessor_DDI-DDS_MC3,assessor_DDI-DDS_MC4,assessor_DDI-DDS_MC5,assessor_DDI-DDS_MC6,assessor_DDI-DDS_MC7,assessor_DDI-DDS_MC8,assessor_DDI-DDS_MC9,assessor_DDI-DDS_MC10,clinical_incident_radio,student_feedback,student_FS-DDS_MC1,student_FS-DDS_MC2,assessor_FS-DDS_MC1,assessor_FS-DDS_MC2,assessor_SOP-DDS_MC1,assessor_SOP-DDS_MC2,assessor_SOP-DDS_MC3,assessor_SOP-DDS_MC4,assessor_SOP-DDS_MC5,assessor_SOP-DDS_MC6,assessor_SOP-DDS_MC7,assessor_SOP-DDS_MC8,student_161_MC1,student_161_MC2,student_161_MC3,student_161_MC4,student_161_MC5,student_DDI-DDS_MC1,student_DDI-DDS_MC2,student_DDI-DDS_MC3,student_DDI-DDS_MC4,student_DDI-DDS_MC5,student_DDI-DDS_MC6,student_DDI-DDS_MC7,student_DDI-DDS_MC8,student_DDI-DDS_MC9,student_DDI-DDS_MC10,assessor_SOR-DDS_MC1,assessor_SOR-DDS_MC2,student_SOP-DDS_MC1,student_SOP-DDS_MC2,student_SOP-DDS_MC3,student_SOP-DDS_MC4,student_SOP-DDS_MC5,student_SOP-DDS_MC6,student_SOP-DDS_MC7,student_SOP-DDS_MC8,student_SOR-DDS_MC1,student_SOR-DDS_MC2,student_SOR-DDS_MC3,student_SOR-DDS_MC4,student_SOR-DDS_MC5,student_SOR-DDS_MC6,assessor_SOR-DDS_MC3,assessor_SOR-DDS_MC4,assessor_SOR-DDS_MC5,assessor_SOR-DDS_MC6,assessor_positioning_MC1,assessor_positioning_MC2,assessor_161_MC1,assessor_161_MC2,assessor_161_MC3,assessor_161_MC4,assessor_161_MC5,student_DDI-BOH_MC1,student_DDI-BOH_MC2,student_DDI-BOH_MC3,student_DDI-BOH_MC4,student_DDI-BOH_MC5,student_DDI-BOH_MC6,student_DDI-BOH_MC7,student_DDI-BOH_MC8,student_DDI-BOH_MC9,student_DDI-BOH_MC10,assessor_DDI-BOH_MC1,assessor_DDI-BOH_MC2,assessor_DDI-BOH_MC3,assessor_DDI-BOH_MC4,assessor_DDI-BOH_MC5,assessor_DDI-BOH_MC6,assessor_DDI-BOH_MC7,assessor_DDI-BOH_MC8,assessor_DDI-BOH_MC9,assessor_DDI-BOH_MC10,DDI-DDS total,DDI-DDS score,DDI-DDS cutoff,DDI-DDS pass,SOR-DDS total,SOR-DDS score,SOR-DDS cutoff,SOR-DDS pass,SOP-DDS total,SOP-DDS score,SOP-DDS cutoff,SOP-DDS pass,FS-DDS total,FS-DDS score,FS-DDS cutoff,FS-DDS pass,161 total,161 score,161 cutoff,161 pass,DDI-BOH total,DDI-BOH score,DDI-BOH cutoff,DDI-BOH pass,DDI-DDS # Yes,DDI-DDS # No,SOR-DDS # Yes,SOR-DDS # No,SOP-DDS # Yes,SOP-DDS # No,FS-DDS # Yes,FS-DDS # No,161 # Yes,161 # No,DDI-BOH # Yes,DDI-BOH # No,Items
0,Paula Yu,839975,Amy Fam,2025-04-03,DDS1,DENT90141,Simulation,1.0,1.0,1.0,2.0,Wear loupes as part of donning PPE. Operator c...,,,,,,,,,,,,No,I learned to understand that high speed burs c...,Yes,Yes,No,No,Yes,Yes,No,No,No,Yes,Yes,Yes,,,,,,,,,,,,,,,,Not Reviewed,Not Reviewed,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Assessed,Not Reviewed,Not Reviewed,Not Reviewed,Not Reviewed,No,Yes,,,,,,,,,,,,,,,,,,,,,,,,,,0,,0.81,,0,,0.69,,30,0.59,0.86,False,0,0,,,0,0,,,0,0,,,0,0,0,0,5,3,0,0,0,0,0,0,"[SOP-DDS, SOR-DDS, FS-DDS]"
1,Paula Yu,839975,Melinda Johansson,2025-03-28,DDS1,DENT90141,Simulation,3.0,2.0,2.0,2.0,Good effort,,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,It is a great learning opportunity. In particu...,,,,,,,,,,,,,,,,,,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,,,,,,,,,,,,,,,,,,,,,Yes,Yes,,,,,,,,,,,,,,,,,,,,,,,,,,139,1.0,0.81,True,0,,0.69,,0,,0.86,,0,0,,,0,0,,,0,0,,,10,0,0,0,0,0,0,0,0,0,0,0,[DDI-DDS]
2,Aaron Khoo,993809,Vicky Thai,2025-04-03,DDS1,DENT90141,Simulation,4.0,4.0,2.0,2.0,Good placement,Took one glove off to handle workbook but held...,,,,,,,,,,,Yes,Overall was smooth. When placing fissure seal ...,Yes,Yes,Yes,Yes,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Yes,Yes,,,,,,,,,,,,,,,,,,,,,,,,,,0,,0.81,,0,,0.69,,0,,0.86,,0,0,,,0,0,,,0,0,,,0,0,0,0,0,0,0,0,0,0,0,0,[FS-DDS]
3,Aaron Khoo,993809,Vicky Thai,2025-04-03,DDS1,DENT90141,Simulation,4.0,3.0,2.0,2.0,Cavity prep not smooth internally but managed ...,,,,,,,,,,,,No,Initial access went towards periphery of cavit...,,,,,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,,,,,,,,,,,,,,,,Yes,Yes,Yes,No,Yes,No,Yes,Yes,Yes,Yes

In [ ]:
def createStudentReport(df, studentName):
    elements = []
    studentdf = df[df['student_name'] == studentName]
    # sort by date
    studentdf['date'] = pd.to_datetime(studentdf['date'])
    studentdf.sort_values('date', inplace=True)
    studentdf['date'] = studentdf['date'].dt.strftime('%d-%m-%Y')
    # filter out 31 march 2025 and 7 april 2025
    # studentdf = studentdf[studentdf['date'].isin(['31-03-2025', '07-04-2025'])]
    savefolder = f'{folder}\\Student Reports'#\\31Mar-7Apr'
    os.makedirs(savefolder, exist_ok=True)
    doc = SimpleDocTemplate(f'{savefolder}\\{studentName}.pdf', pagesize= pageSize, leftMargin = leftMargin,
                            rightMargin = rightMargin, topMargin = topMargin, bottomMargin = bottomMargin)
    
    # first heading
    elements.append(Paragraph(f'{studentName}', headingStyle))
    elements.append(Spacer(1, 24))

    # get the forms
    for i, (idx, row) in enumerate(studentdf.iterrows()):
        formitems = row['Items']
        type = row['type']
        elements.append(Paragraph(f'Form {i+1}: {row[colDate.lower()]}', subheadingStyle))
        elements.append(Spacer(1, 12))
        # elements.append(Paragraph(f'Role: {row[colRole]}', subsubheadingStyle))
        if len(formitems) > 0:
            elements.append(Spacer(1, 12))
            elements.append(Paragraph('Items Performed', subsubheadingStyleL))
            elements.append(Spacer(1, 12))
            elements.append(Paragraph(', '.join(formitems), tableTextStyleL))
        elements.append(Spacer(1, 12))
        # add other columns
        for col in otherCols:
            if pd.isna(row[col]) or row[col] == 'nan' or row[col] == '':
                continue
            # print(f'{col}: {row[col]}')
            elements.append(Paragraph(f'{otherfullnamedict[col] if col in otherfullnamedict.keys() else col}', smallsubsubheadingStyleL))
            elements.append(Spacer(1, 6))
            # text = row[col]
            print(col, row[col])
            text = row[col].strip().replace('\n', '<br/>')
            # remove _x000D_
            text = re.sub(r'_x000D_', '', text)
            # remove non-ascii characters
            text = re.sub(r'[^\x00-\x7F]+', '', text)
            elements.append(Paragraph(f'{text}', tableTextStyleL))
            elements.append(Spacer(1, 12))


        rubricDf = pd.DataFrame(columns = ['Rubric', 'Score'])
        for col in rubricCols:
            rubricDf = pd.concat([rubricDf, pd.DataFrame({'Rubric': [otherfullnamedict[col]], 'Score': [row[col]]})])
        rubricTable = createTable(rubricDf, 'Rubric Scores', colRatio=[1, 3], customTextCols=[0, 1], tableTextStyle=tableTextStyleSmall)
        # alter top and bottom padding
        table = rubricTable._content[2]
        table.setStyle(TableStyle([
            ('TOPPADDING', (0, 0), (-1, -1), 7),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 7),
        ]))
        elements.append(rubricTable)
        elements.append(Spacer(1, 12))
        # add mc col table
        for item in formitems:
            checklist = fullnamedict[item]
            itemName = checklist['name']
            print(item, type)
            stumcCol = [col for col in studentmcCols if item in col]
            assmcCol = [col for col in assessormcCols if item in col]
            if len(studentmcCols) == 0 or len(assessormcCols) == 0:
                continue
            evaldf = pd.DataFrame(columns = ['Marking Checklist', 'Student', 'Educator', 'Critical'])
            for stucol, educol in zip(stumcCol, assmcCol):
                qno = stucol.split('_')[-1]
                if qno not in checklist['fields'].keys():
                    continue
                markingchecklist = checklist['fields'][qno]
                student = row[stucol]
                educator = row[educol]
                critical = criticalDict[item][qno]
                evaldf = pd.concat([evaldf, pd.DataFrame({'Marking Checklist': [markingchecklist], 'Student': [student], 'Educator': [educator], 'Critical': [critical]})])
            
            evaldf.replace({'Not Assessed': 'NA', 'Not Reviewed': 'NA', 'nan': 'NA'}, inplace=True)
            evalTable = createTable(evaldf, f'Evaluation for {item}: {itemName}', colRatio = [5, 1, 1, 1], customTextCols=[0], cellHighlight=True, tableTextStyle=tableTextStyleSmall)
            # alter top and bottom padding
            table = evalTable._content[2]
            # if it's a table
            if isinstance(table, Table):
                table.setStyle(TableStyle([
                    ('TOPPADDING', (0, 0), (-1, -1), 7),
                    ('BOTTOMPADDING', (0, 0), (-1, -1), 7),
                ]))
            elements.append(evalTable)
            itemscore = row[f'{item} score']
            # itemtotal = row[f'{item} total']
            itemcutoff = cutoffDict.get(item, 'N/A')
            itempass = row[f'{item} pass']
            elements.append(Spacer(1, 12))
            elements.append(Paragraph(f'{itemName} Score: {itemscore}  (Cutoff: {itemcutoff})', smallsubsubheadingStyleL))
            elements.append(Spacer(1, 6))
            elements.append(Paragraph(f'Pass: {itempass}', smallsubsubheadingStyleL))
            elements.append(Spacer(1, 12))
        # add pec table
        # pecdf = pd.DataFrame(columns = ['Marking Checklist', 'Student', 'Educator'])
        # for col in pecCols:
        #     pecdf = pd.concat([pecdf, pd.DataFrame({'Marking Checklist': [fullnamedict[col].split(' - ')[-1]], 'Student': [row[col]], 'Educator': [row[col.replace('#1', '#2')]]})])
        # pecdf.replace({'Not Assessed': 'NA', 'Not Reviewed': 'NA', 'nan': 'NA'}, inplace=True)
        # pecTable = createTable(pecdf, 'Other Marking Checklist', colRatio = [5, 1, 1], customTextCols=[0], cellHighlight=True, tableTextStyle=tableTextStyleSmall)
        # elements.append(pecTable)
        # table = pecTable._content[2]
        # table.setStyle(TableStyle([
        #     ('TOPPADDING', (0, 0), (-1, -1), 7),
        #     ('BOTTOMPADDING', (0, 0), (-1, -1), 7),
        # ]))
        elements.append(PageBreak())
    doc.build(elements)

for studentname in df['student_name'].unique():
    print(studentname)
    createStudentReport(df, studentname)
    # break

Paula Yu
student_id 839975
student_name Paula Yu
assessor_name Melinda Johansson
date 28-03-2025
cohort DDS1
type Simulation
student_feedback It is a great learning opportunity. In particular, I learned that while I was performing dental dam isolation, I must be constantly aware of the feelings of my patient. 
assessor_feedback Good effort
assessor_positioning_MC1 Yes
assessor_positioning_MC2 Yes
Creating table for Rubric Scores
DDI-DDS Simulation
Creating table for Evaluation for DDI-DDS: Dental Dam Isolation (DDS)
student_id 839975
student_name Paula Yu
assessor_name Amy Fam
date 03-04-2025
cohort DDS1
type Simulation
student_feedback I learned to understand that high speed burs can be very sharp and therefore I need to careful when I am creating my initial access. The size of the access needs to be as small as possible while still allows the slow speed round bur to reach the caries. 
assessor_feedback Wear loupes as part of donning PPE. Operator chair too low. Sealant placed too thi